In [1]:
import sys
sys.path.append("../")


import os
%matplotlib inline
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from patsy import dmatrices
from openpyxl import load_workbook

from linearmodels import PanelOLS
pd.set_option('display.max_columns', None)

/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/site-packages/linearmodels/panel/data.py:10: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import (Categorical, DataFrame, Index, MultiIndex, Panel, Series,


In [2]:
def coef_with_stars(coef, pvalue):
    coef = round(coef, 2)
    if pvalue >.05:
        coef = str(coef)
    if pvalue <= .05:
        coef = str(coef) + '*'
    if pvalue <= .01:
        coef = coef + '*'
    if pvalue <= .001:
        coef = coef + '*'
    return(coef)
test = coef_with_stars(9.1568, 0.8523)
test

'9.16'

In [3]:
def bonferroni(n_tests, coef, pvalue):
    coef = round(coef, 2)
    if pvalue >(.05/n_tests):
        coef = str(coef)
    if pvalue <= (.05/n_tests):
        coef = str(coef) + '*'
    if pvalue <= (.01/n_tests):
        coef = coef + '*'
    if pvalue <= (.001/n_tests):
        coef = coef + '*'
    return(coef)
test = bonferroni(4, .1, .005)
test

'0.1*'

In [4]:
def format_se(se):
    if se < .005:
        se = '(0.00)'
    else:
        se = '(' + str(round(se, 2)) + ')'
    return se
test = format_se(.0053)
test

'(0.01)'

In [5]:
data_path = '/Users/kylieleblancKylie/domino/dofis/data/'
table_path = '/Users/kylieleblancKylie/domino/dofis/results/Who Needs Rules/'
data = pd.read_csv(os.path.join(data_path, 'clean', 'gdid_subject.csv'),
                  sep=",", low_memory= False)
#load(data)
print(data[data.doi == True].district.nunique())
data.sample(5)

794


,Unnamed: 0,campus,year,test,score,score_std,campname,campischarter,district,distname,distischarter,rating_academic,rating_financial,rating_academic_c,type,type_description,cntyname,students_amind_num,students_asian_num,students_black_num,students_cte_num,students_ell_num,students_frpl_num,students_hisp_num,students_num,students_num_d,students_paci_num,students_sped_num,students_tworaces_num,students_white_num,teachers_badegree_num,teachers_exp_ave,teachers_msdegree_num,teachers_new_num,teachers_nodegree_num,teachers_num,teachers_phddegree_num,teachers_tenure_ave,teachers_turnover_ratio_d,alg_avescore,bio_avescore,eng1_avescore,eng2_avescore,m_3rd_avescore,m_4th_avescore,m_5th_avescore,m_6th_avescore,m_7th_avescore,m_8th_avescore,r_3rd_avescore,r_4th_avescore,r_5th_avescore,r_6th_avescore,r_7th_avescore,r_8th_avescore,s_8th_avescore,us_avescore,alg_numtakers,bio_numtakers,eng1_numtakers,eng2_numtakers,m_3rd_numtakers,m_4th_numtakers,m_5th_numtakers,m_6th_numtakers,m_7th_numtakers,m_8th_numtakers,r_3rd_numtakers,r_4th_numtakers,r_5th_numtakers,r_6th_numtakers,r_7th_numtakers,r_8th_numtakers,s_8th_numtakers,us_numtakers,days_max,days_mean,days_min,class_size_1,class_size_2,class_size_3,class_size_4,class_size_5,class_size_6,class_size_k,class_size_sec_lang,class_size_sec_math,class_size_sec_r,class_size_sec_sci,class_size_sec_ss,stu_teach_ratio,Unnamed: 0_x,link,p_doi,doi_date,total,doi_year,_merge_x,doi,Unnamed: 0_y,_merge_y,cert_area_elem,cert_area_voc,cert_secondary_ela,cert_secondary_math,cert_secondary_sci,certification,certification_report,vocational,county,cnty_pop,students_frpl,students_black,students_hisp,students_white,students_ell,students_sped,students_cte,students_teacher_ratio,geography,charter,district_status,type_urban,type_suburban,type_town,type_rural,eligible,teachers_nodegree,teachers_badegree,teachers_msdegree,teachers_phddegree,r_3rd_std,m_3rd_std,r_4th_std,m_4th_std,r_5th_std,m_5th_std,r_6th_std,m_6th_std,r_7th_std,m_7th_std,r_8th_std,m_8th_std,s_8th_std,alg_std,bio_std,eng1_std,eng2_std,us_std,elem_math,elem_reading,elem,middle_math,middle_reading,middle_science,algebra,biology,eng1,avescores,treatpost,yearpost,yearpre,pre5,pre4,pre3,pre2,pre1,post1,post2,post3,test_by_year,math,reading
180641,206817,165901132,2016,r_6th_avescore,1571.0,-0.384737,BARBARA YARBROUGH EL,N,165901,MIDLAND ISD,N,M,Pass,M,C,OTHER CENTRAL CITY,MIDLAND,6,8,39,0,6,316,393,716,24518,4,37,16,250,31.9,7.2,6.0,3,0.0,37.9,0.0,5.3,20.7,NaN,NaN,NaN,NaN,1410.0,1467.0,1568.0,1567.0,NaN,NaN,1407.0,1439.0,1543.0,1571.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,92.0,84.0,81.0,85.0,NaN,NaN,92.0,83.0,81.0,85.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21,22.6,20,19.2,28.3,28.7,21.4,.,.,.,.,.,18.9,498.0,https://www.midlandisd.net//cms/lib/TX01000898...,0.999955,2018-08-01,10.0,2019.0,both,True,6212.0,left_only,1.000000,NaN,NaN,NaN,NaN,1.0,1.0,NaN,MIDLAND,162.0,0.441341,0.054469,0.548883,0.349162,0.008380,0.051676,0.000000,18.891821,Urban,False,doi,1,0,0,0,1,0.000000,0.841689,0.158311,0.0,-0.352912,-0.531630,-1.141140,-0.979890,-0.163431,-0.398567,-0.294175,-0.761173,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.636696,-0.552494,-0.594595,-0.761173,-0.294175,NaN,NaN,NaN,NaN,-0.577865,False,0.0,-3.0,0,0,1,0,0,0,0,0,r_6th_avescore2016,0,1
17955,20956,113903002,2013,m_7th_avescore,1665.0,0.429399,LOVELADY J H H S,N,113903,LOVELADY ISD,N,M,NaN,M,H,RURAL,HOUSTON,0,1,13,153,0,73,14,223,485,0,27,5,190,NaN,21.4,NaN,1,NaN,23.8,NaN,8.2,8.4,3796.0,4037.0,NaN,3780.0,NaN,NaN,NaN,NaN,1665.0,1691.0,NaN,NaN,NaN,NaN,1667.0,1672.0,3706.0,NaN,49.0,49.0,52.0,32.0,NaN,NaN,NaN,NaN,29.0,43.0,NaN,NaN,NaN,NaN,29.0,46.0,46.0,NaN,NaN,NaN,NaN,.,.,.,.,.,.,.,9,10,11.2,13,13.2,9.4,460.0,http://www.loveladyisd.net/files/user/3/file/L...,0.007645,2018-08-01,6.0,2019.0,both,True,5204.0,left_only,NaN,NaN,0.111111,0.166667,0.0,1.0,1.0,NaN,HOUSTON,23.0,0.327354,0.058296,0.062780,0.852018,0.000000,0.121076,0.686099,9.369748,Rural,False,doi,0,0,0,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.564557,0.473

# Create HTE Variables

In [6]:
data_pre = data.loc[data.pre1 == 1]
data_pre['avescores_pre'] = data_pre.groupby('campus')['score_std'].transform('mean')
data_pre = data_pre.drop_duplicates(subset = 'campus')
data_pre = data_pre.rename(columns = {'students_hisp': 'students_hisp_pre',
                                     'teachers_turnover_ratio_d': 'teachers_turnover_pre',
                                     'students_ell': 'students_ell_pre',
                                      'students_white': 'students_white_pre',
                                      'students_black': 'students_black_pre',
                                     'students_sped': 'students_sped_pre',
                                     'score_std': 'score_std_pre',
                                    'students_num': 'students_num_pre'})
for var in ['students_hisp_pre', 'students_ell_pre', 'students_white_pre', 'students_black_pre',
            'students_num_pre', 'teachers_turnover_pre', 'avescores_pre']:
    for p in [.25, .5, .75, 1]:
        num = str(int(p * 100))
        newvar = var + num
        if p == .25:
            data_pre[newvar] = np.where(data_pre[var] <= data_pre[var].quantile(p), 1, 0)
        if p > .25:
            lp = p - .25
            data_pre[newvar] = np.where(((data_pre[var] > data_pre[var].quantile(lp)) &
                                         (data_pre[var] <= data_pre[var].quantile(p))), 1, 0)
variables = ['campus']
variables = variables + (list(data_pre.filter(regex = ("_pre"))))
data_pre = data_pre[variables]
data_pre = data_pre.merge(data[data.year == 2016][['campus', 'type_urban', 'type_suburban', 'type_town', 'type_rural']], 
                          how = 'left', left_on = ['campus'], right_on = ['campus'])
data_pre = data_pre.rename(columns = {'type_urban': 'type_urban_pre',
                                      'type_suburban': 'type_suburban_pre',
                                      'type_town': 'type_town_pre',
                                      'type_rural': 'type_rural_pre'})
data_pre['teachers_turnover_pre'] = data_pre.teachers_turnover_pre / 100
data_pre.sample(5)

/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


,campus,score_std_pre,students_num_pre,teachers_turnover_pre,students_black_pre,students_hisp_pre,students_white_pre,students_ell_pre,students_sped_pre,avescores_pre,students_hisp_pre25,students_hisp_pre50,students_hisp_pre75,students_hisp_pre100,students_ell_pre25,students_ell_pre50,students_ell_pre75,students_ell_pre100,students_white_pre25,students_white_pre50,students_white_pre75,students_white_pre100,students_black_pre25,students_black_pre50,students_black_pre75,students_black_pre100,students_num_pre25,students_num_pre50,students_num_pre75,students_num_pre100,teachers_turnover_pre25,teachers_turnover_pre50,teachers_turnover_pre75,teachers_turnover_pre100,avescores_pre25,avescores_pre50,avescores_pre75,avescores_pre100,type_urban_pre,type_suburban_pre,type_town_pre,type_rural_pre
28356,33904102,0.678986,219,0.216,0.009132,0.178082,0.785388,0.018265,0.068493,0.757688,1,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0.0,0.0,0.0,1.0
6021,68901047,0.306576,1635,0.170,0.012844,0.900917,0.080122,0.266667,0.083180,-1.238250,0,0,0,1,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,1.0,0.0,0.0,0.0
3285,227901052,2.421100,1365,0.151,0.060073,0.303297,0.480586,0.096703,0.075458,1.085426,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,1,1.0,0.0,0.0,0.0
17107,220905121,-0.008392,335,0.140,0.182090,0.680597,0.101493,0.388060,0.044776,-1.057423,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,1,1,0,0,0,0,1,0,0,1,0,0,0,1.0,0.0,0.0,0.0
384,20901047,1.305564,880,0.128,0.310227,0.392045,0.229545,0.096591,0.125000,0.100960,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,1,0,0.0,1.0,0.0,0.0


In [8]:
data = data.reset_index()
data_pre = data_pre.reset_index()
data = data.reset_index().set_index(['campus', 'test'])
data_pre = data_pre.reset_index().set_index(['campus'])
data.head()

level_0  index  Unnamed: 0  year   score  score_std  \
campus    test                                                                  
109901001 alg_avescore          0      0           0  2012  4069.0  -0.002635   
          bio_avescore          1      1           1  2012  3852.0  -0.712471   
          eng1_avescore         2      2           2  2012  4076.0   0.501276   
          m_3rd_avescore        3      3           3  2012  1463.0   0.269832   
          m_4th_avescore        4      4           4  2012  1494.0  -0.590598   

                               campname campischarter  district    distname  \
campus    test                                                                
109901001 alg_avescore    ABBOTT SCHOOL             N    109901  ABBOTT ISD   
          bio_avescore    ABBOTT SCHOOL             N    109901  ABBOTT ISD   
          eng1_avescore   ABBOTT SCHOOL             N    109901  ABBOTT ISD   
          m_3rd_avescore  ABBOTT SCHOOL             N    109901  ABBOTT ISD   
          m_4th_avescore  ABBOTT SCHOOL             N    109901  ABBOTT ISD   

                         distischarter rating_academic rating_financial  \
campus    test                                                            
109901001 alg_avescore               N             NaN              NaN   
          bio_avescore               N             NaN              NaN   
          eng1_avescore              N             NaN              NaN   
          m_3rd_avescore             N             NaN              NaN   
          m_4th_avescore             N             NaN              NaN   

                         rating_academic_c type type_description cntyname  \
campus    test                                                              
109901001 alg_avescore                 NaN    H            RURAL     HILL   
          bio_avescore                 NaN    H            RURAL     HILL   
          eng1_avescore                NaN    H            RURAL     HILL   
          m_3rd_avescore               NaN    H            RURAL     HILL   
          m_4th_avescore               NaN    H            RURAL     HILL   

                          students_amind_num  students_asian_num  \
campus    test                                                     
109901001 alg_avescore                     0                   0   
          bio_avescore                     0                   0   
          eng1_avescore                    0                   0   
          m_3rd_avescore                   0                   0   
          m_4th_avescore                   0                   0   

                          students_black_num  students_cte_num  \
campus    test                                                   
109901001 alg_avescore                     1               115   
          bio_avescore                     1               115   
          eng1_avescore                    1               115   
          m_3rd_avescore                   1               115   
          m_4th_avescore                   1               115   

                          students_ell_num  students_frpl_num  \
campus    test                                                  
109901001 alg_avescore                   1                101   
          bio_avescore                   1                101   
          eng1_avescore                  1                101   
          m_3rd_avescore                 1                101   
          m_4th_avescore                 1                101   

                          students_hisp_num  students_num  students_num_d  \
campus    test                                                              
109901001 alg_avescore                   30           296             296   
          bio_avescore                   30           296             296   
          eng1_avescore                  30           296             296   
          m_3rd_avescore                 30           296

In [9]:
data_hte = data.reset_index().merge(data_pre, left_on = 'campus', right_on = 'campus', how = 'left')
data_hte.head(5)

,campus,test,level_0_x,index_x,Unnamed: 0,year,score,score_std,campname,campischarter,district,distname,distischarter,rating_academic,rating_financial,rating_academic_c,type,type_description,cntyname,students_amind_num,students_asian_num,students_black_num,students_cte_num,students_ell_num,students_frpl_num,students_hisp_num,students_num,students_num_d,students_paci_num,students_sped_num,students_tworaces_num,students_white_num,teachers_badegree_num,teachers_exp_ave,teachers_msdegree_num,teachers_new_num,teachers_nodegree_num,teachers_num,teachers_phddegree_num,teachers_tenure_ave,teachers_turnover_ratio_d,alg_avescore,bio_avescore,eng1_avescore,eng2_avescore,m_3rd_avescore,m_4th_avescore,m_5th_avescore,m_6th_avescore,m_7th_avescore,m_8th_avescore,r_3rd_avescore,r_4th_avescore,r_5th_avescore,r_6th_avescore,r_7th_avescore,r_8th_avescore,s_8th_avescore,us_avescore,alg_numtakers,bio_numtakers,eng1_numtakers,eng2_numtakers,m_3rd_numtakers,m_4th_numtakers,m_5th_numtakers,m_6th_numtakers,m_7th_numtakers,m_8th_numtakers,r_3rd_numtakers,r_4th_numtakers,r_5th_numtakers,r_6th_numtakers,r_7th_numtakers,r_8th_numtakers,s_8th_numtakers,us_numtakers,days_max,days_mean,days_min,class_size_1,class_size_2,class_size_3,class_size_4,class_size_5,class_size_6,class_size_k,class_size_sec_lang,class_size_sec_math,class_size_sec_r,class_size_sec_sci,class_size_sec_ss,stu_teach_ratio,Unnamed: 0_x,link,p_doi,doi_date,total,doi_year,_merge_x,doi,Unnamed: 0_y,_merge_y,cert_area_elem,cert_area_voc,cert_secondary_ela,cert_secondary_math,cert_secondary_sci,certification,certification_report,vocational,county,cnty_pop,students_frpl,students_black,students_hisp,students_white,students_ell,students_sped,students_cte,students_teacher_ratio,geography,charter,district_status,type_urban,type_suburban,type_town,type_rural,eligible,teachers_nodegree,teachers_badegree,teachers_msdegree,teachers_phddegree,r_3rd_std,m_3rd_std,r_4th_std,m_4th_std,r_5th_std,m_5th_std,r_6th_std,m_6th_std,r_7th_std,m_7th_std,r_8th_std,m_8th_std,s_8th_std,alg_std,bio_std,eng1_std,eng2_std,us_std,elem_math,elem_reading,elem,middle_math,middle_reading,middle_science,algebra,biology,eng1,avescores,treatpost,yearpost,yearpre,pre5,pre4,pre3,pre2,pre1,post1,post2,post3,test_by_year,math,reading,level_0_y,index_y,score_std_pre,students_num_pre,teachers_turnover_pre,students_black_pre,students_hisp_pre,students_white_pre,students_ell_pre,students_sped_pre,avescores_pre,students_hisp_pre25,students_hisp_pre50,students_hisp_pre75,students_hisp_pre100,students_ell_pre25,students_ell_pre50,students_ell_pre75,students_ell_pre100,students_white_pre25,students_white_pre50,students_white_pre75,students_white_pre100,students_black_pre25,students_black_pre50,students_black_pre75,students_black_pre100,students_num_pre25,students_num_pre50,students_num_pre75,students_num_pre100,teachers_turnover_pre25,teachers_turnover_pre50,teachers_turnover_pre75,teachers_turnover_pre100,avescores_pre25,avescores_pre50,avescores_pre75,avescores_pre100,type_urban_pre,type_suburban_pre,type_town_pre,type_rural_pre
0,109901001,alg_avescore,0,0,0,2012,4069.0,-0.002635,ABBOTT SCHOOL,N,109901,ABBOTT ISD,N,NaN,NaN,NaN,H,RURAL,HILL,0,0,1,115,1,101,30,296,296,0,37,9,256,NaN,17.541667,NaN,0,NaN,22.9742,NaN,13.75,8.174,4069.0,3852.0,4076.0,NaN,1463.0,1494.0,1545.0,1700.0,1688.0,1693.0,1466.0,1512.0,1539.0,1647.0,1690.0,1699.0,3808.0,NaN,26.0,26.0,26.0,NaN,16.0,22.0,20.0,17.0,15.0,26.0,16.0,22.0,21.0,19.0,16.0,26.0,26.0,NaN,NaN,NaN,NaN,21,21,17,22,20.6,11.6,20,9.3,11.7,8.5,12.7,16.2,12.884018,0.0,https://www.abbottisd.org/ourpages/auto/2018/3...,0.999955,2018-03-01,11.0,2018.0,both,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HILL,36.0,0.341216,0.003378,0.101351,0.864865,0.003378,0.125,0.388514,12.884018,Rural,False,doi,0,0,0,1,1,NaN,NaN,NaN,NaN,0.712273,0.333386,0.093541,-0.545061,-0.232009,-0.767251,1.064115,1.31147,0.993662,0.843115,0.51854,0.774769,0.159776,0.088879,-0.592073,0.577864,NaN,NaN,-0.326309,0.191269,-0.06752,0.976451,0.

In [10]:
#convert year to datetime
df = data_hte.reset_index()
df['year'] = pd.to_datetime(df['year'], format='%Y')
#add column year to index
df = df.set_index(['year', 'campus'])
#swap indexes
df.index = df.index.swaplevel(0,1)
df[['district', 'doi_year','treatpost']].sample(5, random_state = 8)

,,district,doi_year,treatpost
campus,year,,,
15910127,2017-01-01,15910,2019.0,False
57905268,2014-01-01,57905,2018.0,False
146905101,2019-01-01,146905,2018.0,True
101919043,2013-01-01,101919,2018.0,False
101920114,2014-01-01,101920,2017.0,False


In [12]:
gdid_model = 'score_std ~ + 1 + treatpost + C(test_by_year) + EntityEffects'
linear_gdid_model = 'score_std ~ + 1 + treatpost + yearpost + yearpre  + C(test_by_year) + EntityEffects'
event_study_model = 'score_std ~ + 1 + pre5 + pre4 + pre3 + pre2 + post1 + post2 + post3  + C(test_by_year) + EntityEffects'

In [13]:
def hte_table(covar, htevar, column, start_row, df):
    formula_vars = ''
    covars = ''
    htevars = ''
    hte_list = []
    for p in [25, 50, 75, 100]:
        newvar = htevar + str(p)
        var = covar + str(p)
        if p != 25:
            covars = covars + ' + ' + var
            htevars = htevars + ' + ' + newvar
            hte_list.append(newvar)
        df[newvar] = df['treatpost'] * df[var]

    mod = PanelOLS.from_formula(gdid_model + htevars, df)
    res = mod.fit(cov_type='clustered', clusters = df.district)
    row = start_row
    betas = ['treatpost[T.True]'] + hte_list
    for var in betas:
        ws.cell(row= row, column= column).value = bonferroni(5, res.params[var], res.pvalues[var])
        row = row + 1
        ws.cell(row= row, column= column).value = format_se(res.std_errors[var])
        row = row + 1
    wb.save(file)
    print(res)

In [15]:
def results_col(data, start_row, col):
    # Linear GDID
    mod = PanelOLS.from_formula(linear_gdid_model, data)
    res = mod.fit(cov_type='clustered', clusters = data.district)
    print(res)
    row = start_row
    ws.cell(row= row, column= col).value = coef_with_stars(res.params['treatpost[T.True]'], res.pvalues['treatpost[T.True]'])
    row = row + 1
    ws.cell(row= row, column= col).value = format_se(res.std_errors['treatpost[T.True]'])
    row = row + 1
    ws.cell(row= row, column= col).value = coef_with_stars(res.params['yearpost'], res.pvalues['yearpost'])
    row = row + 1
    ws.cell(row= row, column= col).value = format_se(res.std_errors['yearpost'])
    row = row + 1
    ws.cell(row= row, column= col).value = coef_with_stars(res.params['yearpre'], res.pvalues['yearpre'])
    row = row + 1
    ws.cell(row= row, column= col).value = format_se(res.std_errors['yearpre'])

In [18]:
# Get table ready
file = table_path + 'table7_gdid_linear_math.xlsx'
wb = load_workbook(file)
ws = wb.active

In [23]:
results_col(df[(df.math == 1) & (df.students_hisp_pre25 == 1)], 4, 3)
results_col(df[(df.math == 1) & (df.students_hisp_pre50 == 1)], 4, 4)
results_col(df[(df.math == 1) & (df.students_hisp_pre75 == 1)], 4, 5)
results_col(df[(df.math == 1) & (df.students_hisp_pre100 == 1)], 4, 6)

results_col(df[(df.math == 1) & (df.students_black_pre25 == 1)], 11, 3)
results_col(df[(df.math == 1) & (df.students_black_pre50 == 1)], 11, 4)
results_col(df[(df.math == 1) & (df.students_black_pre75 == 1)], 11, 5)
results_col(df[(df.math == 1) & (df.students_black_pre100 == 1)], 11, 6)

results_col(df[(df.math == 1) & (df.avescores_pre25 == 1)], 18, 3)
results_col(df[(df.math == 1) & (df.avescores_pre50 == 1)], 18, 4)
results_col(df[(df.math == 1) & (df.avescores_pre75 == 1)], 18, 5)
results_col(df[(df.math == 1) & (df.avescores_pre100 == 1)], 18, 6)

results_col(df[(df.math == 1) & (df.teachers_turnover_pre25 == 1)], 25, 3)
results_col(df[(df.math == 1) & (df.teachers_turnover_pre50 == 1)], 25, 4)
results_col(df[(df.math == 1) & (df.teachers_turnover_pre75 == 1)], 25, 5)
results_col(df[(df.math == 1) & (df.teachers_turnover_pre100 == 1)], 25, 6)

results_col(df[(df.math == 1) & (df.type_urban_pre == 1)], 32, 3)
results_col(df[(df.math == 1) & (df.type_suburban_pre == 1)], 32, 4)
results_col(df[(df.math == 1) & (df.type_town_pre == 1)], 32, 5)
results_col(df[(df.math == 1) & (df.type_rural_pre == 1)], 32, 6)

wb.save(file)

                          PanelOLS Estimation Summary                           
Dep. Variable:              score_std   R-squared:                        0.1676
Estimator:                   PanelOLS   R-squared (Between):             -0.1276
No. Observations:              217872   R-squared (Within):               0.1676
Date:                Tue, Jan 14 2020   R-squared (Overall):              0.0134
Time:                        17:50:48   Log-likelihood                -1.921e+05
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      751.06
Entities:                        5950   P-value                           0.0000
Avg Obs:                       36.617   Distribution:               F(58,216404)
Min Obs:                       0.0000                                           
Max Obs:                       840.00   F-statistic (robust):             50.950
                            

                          PanelOLS Estimation Summary                           
Dep. Variable:              score_std   R-squared:                        0.1993
Estimator:                   PanelOLS   R-squared (Between):             -0.5048
No. Observations:              203142   R-squared (Within):               0.1993
Date:                Tue, Jan 14 2020   R-squared (Overall):             -0.0119
Time:                        17:50:55   Log-likelihood                 -1.67e+05
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      865.46
Entities:                        5953   P-value                           0.0000
Avg Obs:                       34.124   Distribution:               F(58,201674)
Min Obs:                       0.0000                                           
Max Obs:                       840.00   F-statistic (robust):             54.829
                            

                          PanelOLS Estimation Summary                           
Dep. Variable:              score_std   R-squared:                        0.2460
Estimator:                   PanelOLS   R-squared (Between):             -0.7837
No. Observations:              197219   R-squared (Within):               0.2460
Date:                Tue, Jan 14 2020   R-squared (Overall):              0.0300
Time:                        17:51:02   Log-likelihood                -1.569e+05
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      1101.1
Entities:                        5952   P-value                           0.0000
Avg Obs:                       33.135   Distribution:               F(58,195753)
Min Obs:                       0.0000                                           
Max Obs:                       840.00   F-statistic (robust):             51.369
                            

                          PanelOLS Estimation Summary                           
Dep. Variable:              score_std   R-squared:                        0.3336
Estimator:                   PanelOLS   R-squared (Between):             -0.8644
No. Observations:              193028   R-squared (Within):               0.3336
Date:                Tue, Jan 14 2020   R-squared (Overall):              0.0719
Time:                        17:51:08   Log-likelihood                -1.451e+05
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      1653.2
Entities:                        5961   P-value                           0.0000
Avg Obs:                       32.382   Distribution:               F(58,191560)
Min Obs:                       0.0000                                           
Max Obs:                       840.00   F-statistic (robust):             118.79
                            

                          PanelOLS Estimation Summary                           
Dep. Variable:              score_std   R-squared:                        0.1917
Estimator:                   PanelOLS   R-squared (Between):             -0.1857
No. Observations:              244329   R-squared (Within):               0.1917
Date:                Tue, Jan 14 2020   R-squared (Overall):              0.0654
Time:                        17:51:16   Log-likelihood                -2.315e+05
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      992.86
Entities:                        5961   P-value                           0.0000
Avg Obs:                       40.988   Distribution:               F(58,242862)
Min Obs:                       0.0000                                           
Max Obs:                       840.00   F-statistic (robust):             49.721
                            

...

                          PanelOLS Estimation Summary                           
Dep. Variable:              score_std   R-squared:                        0.1733
Estimator:                   PanelOLS   R-squared (Between):             -0.3925
No. Observations:              202444   R-squared (Within):               0.1733
Date:                Tue, Jan 14 2020   R-squared (Overall):              0.0756
Time:                        17:52:25   Log-likelihood                -1.852e+05
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      727.00
Entities:                        5961   P-value                           0.0000
Avg Obs:                       33.961   Distribution:               F(58,201123)
Min Obs:                       0.0000                                           
Max Obs:                       840.00   F-statistic (robust):             39.538
                            

                          PanelOLS Estimation Summary                           
Dep. Variable:              score_std   R-squared:                        0.2929
Estimator:                   PanelOLS   R-squared (Between):             -0.4934
No. Observations:              224225   R-squared (Within):               0.2929
Date:                Tue, Jan 14 2020   R-squared (Overall):             -0.0059
Time:                        17:52:33   Log-likelihood                -1.654e+05
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      1589.8
Entities:                        5779   P-value                           0.0000
Avg Obs:                       38.800   Distribution:               F(58,222574)
Min Obs:                       0.0000                                           
Max Obs:                       784.00   F-statistic (robust):         -2.402e+14
                            

                          PanelOLS Estimation Summary                           
Dep. Variable:              score_std   R-squared:                        0.2938
Estimator:                   PanelOLS   R-squared (Between):             -0.8317
No. Observations:              314544   R-squared (Within):               0.2938
Date:                Tue, Jan 14 2020   R-squared (Overall):             -0.1128
Time:                        17:52:43   Log-likelihood                 -2.17e+05
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      2238.5
Entities:                        5959   P-value                           0.0000
Avg Obs:                       52.785   Distribution:               F(58,312017)
Min Obs:                       0.0000                                           
Max Obs:                       517.00   F-statistic (robust):             137.17
                            

                          PanelOLS Estimation Summary                           
Dep. Variable:              score_std   R-squared:                        0.2974
Estimator:                   PanelOLS   R-squared (Between):             -0.9638
No. Observations:               94871   R-squared (Within):               0.2974
Date:                Tue, Jan 14 2020   R-squared (Overall):              0.0549
Time:                        17:52:47   Log-likelihood                -6.897e+04
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      685.68
Entities:                        5949   P-value                           0.0000
Avg Obs:                       15.947   Distribution:                F(58,93963)
Min Obs:                       0.0000                                           
Max Obs:                       840.00   F-statistic (robust):             48.454
                            

                          PanelOLS Estimation Summary                           
Dep. Variable:              score_std   R-squared:                        0.1652
Estimator:                   PanelOLS   R-squared (Between):             -0.0427
No. Observations:              177355   R-squared (Within):               0.1652
Date:                Tue, Jan 14 2020   R-squared (Overall):              0.0946
Time:                        17:52:52   Log-likelihood                 -1.84e+05
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      602.48
Entities:                        5961   P-value                           0.0000
Avg Obs:                       29.753   Distribution:               F(58,176611)
Min Obs:                       0.0000                                           
Max Obs:                       840.00   F-statistic (robust):             42.664
                            